<a href="https://colab.research.google.com/github/zimbit78/IAA2_Jalon2/blob/main/Jalon2_Thomas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
#pour choisir les topics :
from sklearn.decomposition import NMF

In [ ]:
DATASET_FILE = "./dataset_tokens.csv"
dataset_df = pd.read_csv(DATASET_FILE)
dataset_df

,Unnamed: 0,text,stars,length,tokens
0,0,I've only had food from here once and it wasn'...,1,68,NOT_memorable panang curry balance flavor NOT_...
1,1,I will never return here again. Ever. I was ...,1,87,NOT_return ever sit booth wait dinner come scu...
2,2,I wish my experience was great as others. I di...,1,166,wish experience others din wednesday night wee...
3,3,Are the rosemary grapefruit scones supposed to...,1,81,rosemary grapefruit scone suppose taste whole ...
4,4,Our takeout order was half wrong. Food was mis...,1,32,takeout order half wrong miss portion size wro...
...,...,...,...,...,...
24995,24995,I was a loyal fan of Aroy before the ownership...,5,75,loyal fan aroy ownership change apprehensive v...
24996,24996,Stopped here for a bite while wandering around...,5,55,stop bite wander around faneuil hall pleasantl...
24997,24997,"A quiet place with excellent food, great music...",5,32,quiet excellent music helpful wait staff time ...
24998,24998,Super delicious food. Awesome vibe. I suffered...,5,41,super delicious awesome vibe suffer disneyworl...


In [ ]:
Comm_lst = dataset_df['text'][dataset_df['stars'] < 3].values
tokens_lst = dataset_df['tokens'][dataset_df['stars'] < 3].values

In [ ]:
vectorizer = TfidfVectorizer(max_df=.8, min_df=.007)
#.8 et .001
X = vectorizer.fit_transform(tokens_lst)
df = pd.DataFrame(data=X.toarray(), columns=vectorizer.get_feature_names_out())
df.head()

,00,10,100,11,12,15,18,20,25,2nd,30,40,45,50,able,absolutely,accommodate,acknowledge,across,act,actual,actually,add,additional,afternoon,ago,agree,ahead,alcohol,all,allow,almost,alone,along,already,alright,also,although,always,amaze,...,weird,welcome,well,whatever,whether,which,white,whole,wife,willing,window,wine,wing,wish,within,without,woman,wonder,wonderful,word,work,worker,world,worst,worth,would,wow,wrap,write,wrong,yeah,year,yell,yelp,yes,yesterday,yet,you,young,zero
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.177724,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.114462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.153643,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.120212,0.000000,0.0,0.0,0.0,0.0,0.0,0.117267,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.197217,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.098418,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.477504,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.118726,0.0,0.0,0.0,0.436554,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def display_topics(model, feature_names, num_top_words, topic_names=None):
    '''Given an NMF model, feature_names, and number of top words, print topic number and its top feature names, up to specified number of top words.'''
    # iterate through topics in topic-term matrix, 'H' aka
    # model.components_
    for ix, topic in enumerate(model.components_):
        #print topic, topic number, and top words
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i] \
             for i in topic.argsort()[:-num_top_words - 1:-1]]))


In [ ]:
n_topic = 15
nmf_model = NMF(n_topic)

# Learn an NMF model for given Document Term Matrix 'V' 
# Extract the document-topic matrix 'W'
W = nmf_model.fit_transform(df)

# Extract top words from the topic-term matrix 'H' 
num_top_words = 15
display_topics(nmf_model, vectorizer.get_feature_names_out(), num_top_words)

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,



Topic  0
say, call, would, tell, could, give, ask, manager, phone, star, room, card, hour, day, hotel

Topic  1
taste, sauce, dish, flavor, bland, really, rice, soup, taco, noodle, beef, thai, shrimp, fish, come

Topic  2
pizza, crust, cheese, topping, slice, delivery, sauce, pie, thin, cold, sausage, soggy, deliver, well, quality

Topic  3
order, take, wrong, delivery, come, drink, minute, get, item, time, min, half, ask, receive, cold

Topic  4
us, table, come, server, waitress, ask, waiter, seat, take, sit, check, water, bring, hostess, leave

Topic  5
burger, fry, bun, onion, well, patty, medium, cheese, shake, cook, bacon, cold, rare, ring, would

Topic  6
wait, minute, hour, long, min, seat, slow, line, half, finally, another, arrive, later, come, still

Topic  7
bar, drink, beer, bartender, night, friend, area, sit, game, music, people, pretty, crowd, sport, nice

Topic  8
chicken, rice, fry, wing, piece, dry, salad, sauce, curry, lunch, general, cook, fried, meat, spicy

Topic

In [ ]:
def sort_index(lst, rev=True):
    index = range(len(lst))
    s = sorted(index, reverse=rev, key=lambda i: lst[i])
    return s

#get phrases from indexes
def ind2text(comments, n) :
  return comments[n]

In [ ]:
#3 best phrases per topic
bestphrases = []
for t in range(n_topic) :
  tps = []
  for ind in sort_index([i[t] for i in W])[:3]:
    tps.append(ind2text(Comm_lst, ind))
  bestphrases.append(tps)

In [ ]:
col1 = []
for ix, topic in enumerate(nmf_model.components_):
  #print topic, topic number, and top words
  col1.append([vectorizer.get_feature_names_out()[i] \
        for i in topic.argsort()[:-num_top_words - 1:-1]])

In [ ]:
table = []
table.append(col1)
for i in range(3) :
  table.append([text[i] for text in bestphrases])

numpy_array = np.array(table)
transpose = numpy_array.T

topics_df = pd.DataFrame(transpose, columns = ['top_features', 'top_review1', 'top_review2', 'top_review3'], index=['topic'+str(i) for i in range(len(col1))])
topics_df

In [ ]:
thisdict = {
  "topic0" : "Bad managment",
  "topic1" : "Disappointment",
  "topic2" : "Bad Pizza",
  "topic3" : "Order problem",
  "topic4" : "Bad service",
  "topic5" : "Bad Burger",
  "topic6" : "Too much wait",
  "topic7" : "Bad bar",
  "topic8" : "Bad Chicken",
  "topic9" : "Bad food - Restaurant",
  "topic10": "Bad Sushi",
  "topic11": "Overpriced or underquantitied",
  "topic12": "Bad customer services",
  "topic13": "Bad Sandwich",
  "topic14": "Changes not appreciated"
}

In [ ]:
import pickle #pour save les fichiers en version binaire

pickle.dump( vectorizer, open( "vectorizer.p", "wb" ) )
pickle.dump( nmf_model, open( "model.p", "wb" ) )
topics_df.to_csv('./dataset_topics.csv')